# Instalación

Para este tutorial vamos a necesitar:

* CPython 3 (creo que todo debería ser compatible con Python 2.7.x pero no lo he comprobado)
* IPython (para poder ejecutar los notebooks)
* Numpy
* numba (en su última versión oficial)
* Cython
* matplotlib (para dibujar alguna cosa)
* PyPy (usaremos una versión compatible con CPython 2.7 por conveniencia)
* Numpypy

Si tenés todo lo anterior instalado podéis saltaros el siguiente paso.

## CPython, numba y Cython

Para toda la parte de Python, numba y Cython vamos a usar la distribución Anaconda con Python 3.4. Esta distribución, además, instala un montón de paquetes útiles para hacer cálculo científico con Python (entre ellos Numpy y Matplotlib).

https://continuum.io/downloads

Hemos preparado unos pendrives con Anaconda 32 bits para Linux y Windows. Para otros sistemas operativos podéis ir a la página oficial de continuum, creadores de Anaconda, y descargaros la versión para vuestro sistema.

## Pypy y numpypy

Para los cálculos que involucren a Pypy y numpypy vamos a usar una versión de pypy portable que contiene, además, a numpypy.

Como en el caso anterior, hemos preparado unos pendrives con la información.

Tendréiis que copiar el archivo comprimido en vuestra máquina, descomprimir donde queráis y, por último, hacer:

`sudo ln -s /path/to/pypy /usr/local/bin`

En windows no es una tarea sencilla instalarlo por lo que si alguno trae un equipo windows lo puede intentar con la información de la página oficial. En Mac no tengo mucha idea. Lo ideal es que la instalación la dejéis para casa y aquí, en esas partes (pequeñas del tutorial), lo ejecutaremos nosotros.

# El problema que vamos a optimizar

Deflexión de una placa rectangular, simplemente apoyada en sus cuatro bordes (es decir, los bordes pueden girar: no están empotrados) sometida a una carga transversal. Este problema tiene solución analítica conocida desde hace tiempo, hallada por Navier:

$$w(x,y) = \sum_{m=1}^\infty \sum_{n=1}^\infty \frac{a_{mn}}{\pi^4 D}\,\left(\frac{m^2}{a^2}+\frac{n^2}{b^2}\right)^{-2}\,\sin\frac{m \pi x}{a}\sin\frac{n \pi y}{b}$$

siendo $a_{mn}$ los coeficientes de Fourier de la carga aplicada. Como veis, para cada punto $(x, y)$ hay que hacer una doble suma en serie; si encima queremos evaluar esto en un `meshgrid`, necesitamos **un cuádruple bucle**. Ya se anticipa que por muy hábiles que estemos, a Python le va a costar.

Una solución que, a priori, podríamos pensar que es óptima dentro del mundo Python sería usando numpy arrays y Python y quedaría de la siguiente forma:

```python
def plate_displacement(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n):
    max_i, max_j = ww.shape
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            for ii in range(max_i):
                for jj in range(max_j):
                    a_mn = 4*P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
                    ww[ii, jj] += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                                   * sin(mm * pi * xx[ii, jj] / a)
                                   * sin(nn * pi * yy[ii, jj] / b)
                                   / (pi**4 * D)) 
```                                  

<div class="alert alert-info">Nótese que el array de desplazamientos <code>ww</code> es un argumento de la función, lo cual quiere decir que lo estamos creando en otra parte del código. Esto nos ayudará a medir el rendimiento de forma más uniforme.</div>

## Definimos un caso de ejemplo

```python
import numpy as np
from numpy import sin, pi

# Plate geometry
a = 1.0  # Length, m
b = 1.0  # Width, m
h = 50e-3  # Thickness, m

# Material properties
E = 69e9  # Elastic modulus, Pa
nu = 0.35  # Poisson's ratio

# Series terms
max_m = 16
max_n = 16

# Computation points
# NOTE: With an odd number of points the center of the place is included in
# the grid
NUM_POINTS = 101

# Load
P = 10e3  # Point load, N
xi = a / 2
eta = a / 2

# Flexural rigidity
D = h**3 * E / (12 * (1 - nu**2))

# ---

# Set up domain
x = np.linspace(0, a, num=NUM_POINTS)
y = np.linspace(0, b, num=NUM_POINTS)
xx, yy = np.meshgrid(x, y)

# Compute displacement field
ww = np.zeros_like(xx)
plate_displacement(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n)

# Print maximum displacement
w_max = abs(ww).max()
print("Maximum displacement = %14.12f mm" % (w_max * 1e3))
print("alpha = %7.5f" % (w_max / (P * a**2 / D)))
print("alpha * P a^2 / D = %6.4f mm" % (0.01160 * P * a**2 / D * 1e3))
```

El resultado que deberíamos obtener de lo antterior sería:

```
Maximum displacement = 0.141317840389 mm
alpha = 0.01158
alpha * P a^2 / D = 0.1416 mm
```

Y para visualizarlo:

```python
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rc('axes.formatter', limits=(-3, 3))

fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(xx, yy, ww, cmap=plt.cm.Spectral)
```

![Placa de Navier](./static/navier_plate.png)

# AVISO IMPORTANTE

<DIV class="alert alert-info">Los números y porcentajes que veáis a continuación pueden variar levemente (o bastante) dependiendo de la máquina donde se ejecute. Tomad los valores como aproximados.</div>